<a href="https://colab.research.google.com/github/alex-jk/SWB-GVCEH/blob/main/models/relevance_model/Relevance_Model_Reddit_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install setfit
!pip install tqdm
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=df15db3b864ab45d1a483943aa11dbcb3c88af19e2f1aa15fa72f91a705a0803
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import pandas as pd
import numpy as np
from setfit import SetFitModel
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
import spacy
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

model_sent_transformer = SentenceTransformer('all-MiniLM-L6-v2')
# Load spaCy model for sentence tokenization
nlp = spacy.load("en_core_web_sm")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

##### Import reddit datasets

In [4]:
file_path = '/content/drive/My Drive/SWB-GVCEH/VictoriaBC_data_updated.csv'
VictoriaBC_data_nodups = pd.read_csv(file_path)
VictoriaBC_data_nodups = VictoriaBC_data_nodups.drop_duplicates().reset_index(drop=True)

print("\nVictoria BC data ----------------")
print(VictoriaBC_data_nodups.shape)
print(VictoriaBC_data_nodups.columns)
print(VictoriaBC_data_nodups.head())

Sooke_data_nodups = pd.read_csv('/content/drive/My Drive/SWB-GVCEH/Sooke_data.csv')
Sooke_data_nodups = Sooke_data_nodups.drop_duplicates().reset_index(drop=True)

print("\nSooke data ----------------")
print(Sooke_data_nodups.shape)
print(Sooke_data_nodups.columns)
print(Sooke_data_nodups.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Victoria BC data ----------------
(8998, 6)
Index(['Subreddit', 'Title', 'Text', 'Submission ID', 'User ID',
       'Search Term'],
      dtype='object')
    Subreddit                                              Title  \
0  VictoriaBC  True change around homelessness from the homel...   
1  VictoriaBC  New transitional housing facility on Douglas o...   
2  VictoriaBC   Where to buy a reasonably priced Christmas tree?   
3  VictoriaBC  Looking for a Christmas tree? Please consider ...   
4  VictoriaBC                              Free clothing places?   

                                                Text Submission ID  \
0  I've started my own organization to expose the...        zen3ao   
1                                                NaN        5yah6z   
2  Went to the Christmas tree farm today and thou...       17yjdiw   
3                                                NaN        7h0e58   
4  Hey so long story short I moved here from Sask...       169ywlj   

          User 

In [5]:
file_url = 'https://drive.google.com/uc?id=1ANE3_UkNi2UGyQpHr8Ujz1QwBNKxmcTl'

# Reading the CSV file into a DataFrame
additional_reddit_df = pd.read_csv(file_url)
additional_reddit_df = additional_reddit_df.drop_duplicates().reset_index(drop=True)
additional_reddit_df.rename(columns={'Body': 'Text'}, inplace=True)

print("\nAdditional reddit data ----------------")
print(additional_reddit_df.shape)
print(additional_reddit_df.columns)
print(additional_reddit_df.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Additional reddit data ----------------
(2057, 4)
Index(['Subreddit', 'Title', 'Text', 'Comments'], dtype='object')
               Subreddit                                              Title  \
0  OakBayBritishColumbia                            Oak Bay high right now.   
1  OakBayBritishColumbia  Food share for Ukrainian refugees on Vancouver...   
2  OakBayBritishColumbia     Lost Budgie - Cadboro Bay (Willows Elementary)   
3  OakBayBritishColumbia  What are 3 things I must see when visiting Oak...   
4  OakBayBritishColumbia  Yes, this is a leaf blower rant, but hear me o...   

                                                Text  \
0  Anyone know what the hell is going on at Oak B...   
1                                                NaN   
2  My friends blue budgie flew out of the house l...   
3                                            Thanks!   
4  I have a neighbour who uses his leaf blower to...   

                                            Comments  
0  ['https://www

##### Combine all data into one df

In [6]:
# Combining the DataFrames
select_cols = ['Subreddit', 'Title', 'Text']
combined_df = pd.concat([VictoriaBC_data_nodups[select_cols], Sooke_data_nodups[select_cols], additional_reddit_df[select_cols]], ignore_index=True)

# Resetting the index
combined_df.reset_index(drop=True, inplace=True)

combined_df['TitleText'] = combined_df['Title'].fillna('') + '. ' + combined_df['Text'].fillna('')

nan_titletext_df = combined_df[combined_df['TitleText'].isna()]
# Resetting the index
nan_titletext_df.reset_index(drop=True, inplace=True)

# Remove rows where 'TitleText' is NaN
combined_df.dropna(subset=['TitleText'], inplace=True)
# Resetting the index again after dropping rows
combined_df.reset_index(drop=True, inplace=True)

print("\nCombined data ----------------")
print(combined_df.shape)
print(combined_df.columns)
print(combined_df.head())


Combined data ----------------
(11160, 4)
Index(['Subreddit', 'Title', 'Text', 'TitleText'], dtype='object')
    Subreddit                                              Title  \
0  VictoriaBC  True change around homelessness from the homel...   
1  VictoriaBC  New transitional housing facility on Douglas o...   
2  VictoriaBC   Where to buy a reasonably priced Christmas tree?   
3  VictoriaBC  Looking for a Christmas tree? Please consider ...   
4  VictoriaBC                              Free clothing places?   

                                                Text  \
0  I've started my own organization to expose the...   
1                                                NaN   
2  Went to the Christmas tree farm today and thou...   
3                                                NaN   
4  Hey so long story short I moved here from Sask...   

                                           TitleText  
0  True change around homelessness from the homel...  
1  New transitional housing facili

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# ind = 0
# print(nan_titletext_df.iloc[ind])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndexError: ignored

##### Load twitter data

In [ ]:
# List of file names
file_names = [
    'GVCEH-tweets-combined_2023-02-08.csv',
    'GVCEH-tweets-combined_2023-01-30.csv',
    'GVCEH-tweets-combined_2023-01-21.csv',
    'GVCEH-tweets-combined_2023-01-12.csv'
]

# Base URL for raw files in the GitHub repository
base_url = 'https://raw.githubusercontent.com/alex-jk/SWB-GVCEH/main/data/processed/twitter/github_actions/'

# Initialize a list to collect the DataFrames
dfs = []

for file_name in file_names:
    # Construct the full URL for the current file
    file_url = base_url + file_name
    # Read the CSV file
    current_df = pd.read_csv(file_url)
    # Append the DataFrame to the list
    dfs.append(current_df)

# Concatenate all DataFrames in the list
tweets_combined_df = pd.concat(dfs, ignore_index=True)
# Remove duplicates
tweets_combined_df = tweets_combined_df.drop_duplicates().reset_index(drop=True)

# Displaying the first few rows of the DataFrame
print(tweets_combined_df.shape)
print(tweets_combined_df.columns)
print(tweets_combined_df.head())

##### Check twitter data

In [ ]:
ind = 8
print(tweets_combined_df['search_neighbourhood'][ind])
print(tweets_combined_df['search_keywords'][ind])
print(tweets_combined_df['text'][ind])

In [ ]:
# Load the pretrained SetFit model
model = SetFitModel.from_pretrained("sheilaflood/gvceh-setfit-rel-model2")

# Example text data
texts = ["Example text relevant to homelessness in Victoria.", "Irrelevant text about other topics."]

# Model makes predictions
predictions = model(texts)
print(predictions)

In [ ]:
ind = 7
model(tweets_combined_df['text'][ind])

##### Check that tweets are considered relevant by the model
- all tweets were found to be relevant by the model

In [ ]:
# Define a function to make predictions
def get_prediction(text):
    prediction = model([text])[0]
    return prediction

# # Apply the model to each row in the 'text' column with a progress bar
# tqdm.pandas()  # Enable tqdm for pandas
# tweets_combined_df['relevant'] = tweets_combined_df['text'].progress_apply(get_prediction)

In [ ]:
def extract_integer_from_tensor(tensor_val):
    return tensor_val.item()

# Apply this function to the entire column
# tweets_combined_df['relevant'] = tweets_combined_df['relevant'].apply(extract_integer_from_tensor)

# Print value counts for the 'relevant' column
# print(tweets_combined_df['relevant'].value_counts())

##### Perform K-Means clustering of twitter data
Generate embeddings of twitter posts

In [ ]:
twitter_embeddings = model_sent_transformer.encode(tweets_combined_df['text'].tolist(), show_progress_bar=True)

##### Run K-Means on twitter embeddings

In [ ]:
# Calculate sum of squared distances for different number of clusters
Sum_of_squared_distances = []
K = range(1,40)  # Adjust the range based on your dataset
for k in K:
    print(f"Current k: {k}")
    km = KMeans(n_clusters=k, n_init=10)
    km = km.fit(twitter_embeddings)
    Sum_of_squared_distances.append(km.inertia_)

# Plot the elbow graph
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of squared distances')
plt.title('Twitter Embeddings Elbow Method For Optimal k')
plt.show()

#####Assign clusters to embeddings

In [ ]:
from sklearn.cluster import KMeans

k = 15  # number of clusters
kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
kmeans.fit(twitter_embeddings)

cluster_centroids = kmeans.cluster_centers_

tweets_combined_df['cluster'] = kmeans.labels_

In [ ]:
print(tweets_combined_df['cluster'].unique())

ind = 2
cluster_tweets = tweets_combined_df[tweets_combined_df['cluster'] == ind].copy().reset_index(drop=True)
for i in range(0, 10):
  print("\n---------------------Printing tweet")
  print(cluster_tweets['text'][i])

In [ ]:
# Function to split text into sentences
def spacy_sentence_split(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

import concurrent.futures

# Function to process a single Reddit post and determine its relevance
def process_post_for_relevance(post, model, centroids, threshold):
    sentences = spacy_sentence_split(post)
    sentence_embeddings = model.encode(sentences, show_progress_bar=False)
    sentence_relevance_scores = cosine_similarity(sentence_embeddings, centroids)
    max_scores = sentence_relevance_scores.max(axis=1)
    relevant_sentences = max_scores > threshold
    relevance_score = relevant_sentences.mean()  # Proportion of sentences above the threshold
    return relevance_score

# Parallel processing function
def parallel_relevance_computation(reddit_posts, model, centroids, threshold, max_workers=10):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        relevance_scores = list(tqdm(executor.map(lambda post: process_post_for_relevance(post, model, centroids, threshold), reddit_posts), total=len(reddit_posts)))
    return relevance_scores

In [ ]:
print(combined_df.shape)
check_post = combined_df['TitleText'][11]

print(check_post)

process_post_for_relevance(check_post, model_sent_transformer, cluster_centroids, threshold=0.5)

In [ ]:
# Apply parallel relevance computation to the DataFrame
combined_df['relevance_score'] = parallel_relevance_computation(combined_df['TitleText'].tolist(), model_sent_transformer, cluster_centroids, threshold=0.5)